In [111]:
import pandas as pd
import re

In [112]:
df = pd.read_csv("filtered_activity_based_EF_database_out.csv")

In [113]:
def to_camel_case(string):
    return "".join(x.capitalize() for x in str(string).lower().split(" "))


def to_lower_camel_case(string):
    # We capitalize the first letter of each component except the first one
    # with the 'capitalize' method and join them together.
    camel_string = to_camel_case(string)
    return str(string)[0].lower() + camel_string[1:]


def formatString(string):
    if pd.isna(string) == False:
        clean_string = "".join(re.findall(r"([a-zA-Z]|\d|\s|\.*)", string))
        return to_lower_camel_case(clean_string)
    # If blank, then in the types, we only want an empty array
    return 


def convertUnit(unit):
    unit_conversions = {
        "kg": "kg",
        "litres": "l",
        "GJ": "GJ",
        "kWh": "kWh",
        "kWh (Net CV)": "kWh",
        "kWh (Gross CV)": "kWhGross",
        "tonnes": "t",
        "Room per night": "roomsPerNight",
        "miles": "miles",
        "km": "km",
        "m3": "m3",
        "passenger-km": "pkm",
        "cubic metres (m3)": "m3",
        "million litres": "Ml",
        "tonne-km": "tkm",
        "FTE working hour": "FTEhour",
    }
    try:
        return unit_conversions[unit]
    except:
        raise Exception(
            f"Unit {unit} was not found in unit_conversions. \nPlease add value that corresponds to the manualEmissions-database unit and ensure that the unit is handled in the app and backend"
        )


# The output of this function should be the format of the values stored in the databases for the activities and categories
# It should also be the format used in the frontend for mapping values to messages.
def printFormattedUniqueValues(df, column):
    print(f"\n======= {column} =======\n")
    print(
        [
            x
            for x in df[column].apply(lambda row: formatString(row)).unique().tolist()
            if x != None
        ]
    )

def printFormattedTypesForActivityL2(df):
    print(f"\n======= Activity L2 types per L1 =======\n")
    for scopeType in df["GHG Scopes and categories L2"].unique():
        for activityL1 in df[df["GHG Scopes and categories L2"]== scopeType]["Activity L1"].unique():
            print(f"\nexport const {formatString(scopeType+' '+activityL1)} = [")
            
            for activityL2 in df[df["GHG Scopes and categories L2"]== scopeType][df["Activity L1"] == activityL1]["Activity L2"].unique():
                print(f"'{formatString(activityL2)}',")
                
            print("] as const;\n")

def printSpreadOfActivityL1(df):
    print(f"\n======= Activity L1 spread =======\n")
    for scopeType in df["GHG Scopes and categories L2"].unique():
        for activityL1 in df[df["GHG Scopes and categories L2"]== scopeType]["Activity L1"].unique():
            print(f"...{formatString(scopeType+' '+activityL1)},")

def printFormattedConstantsForSelectScopeType(df):
    print(f"\n======= Constants for select scope type =======\n")
    for scopeType in df["GHG Scopes and categories L2"].unique():
        print(f"{formatString(scopeType)}: ", "{")
        for activityL1 in df[df["GHG Scopes and categories L2"] == scopeType]["Activity L1"].unique():
            print(f"    {formatString(activityL1)}: ", "{ \n    activityL2: ",f"{formatString(scopeType+' '+activityL1)}","," )
            print(f"    unitOfMeasurement: [...calculatedUnits,", )
            for UOM in df[df["Activity L1"] == activityL1]["Activity unit"].unique():
                print(f"{convertUnit(UOM)},")
            print("] as const, \n    },")
        print("},")
        
def printMessagesForSelectScopeType(df, column):
    print(f"\n======= Messages for {column} =======\n")
    for activity in df[column].unique():
        print(f"'{formatString(activity)}': ", "{")
        print(f"    defaultMessage: '{activity}' ,")
        print(f"    description: 'Label for",f"{column} {formatString(activity)}' ,")
        print("},")
            
      
def printCasesForMessagesHelpers(df, column):
    print(f"\n======= Messages for {column} =======\n")
    for activity in df[column].unique():
        print(f"case '{formatString(activity)}': ")
        if "." in str(activity):
            print(f"    return {formatString(column)}Messages['{formatString(activity)}'];")
        else:
            print(f"    return {formatString(column)}Messages.{formatString(activity)};")
            

In [114]:
printFormattedUniqueValues(df[df["GHG Scopes and categories L1"] == "Scope 3"], "Activity L1")


======= Activity L1 =======

['flights', 'landTravel', 'seaTravel', 'homeOffice', 'buildingMaterials', 'clothingAndFootwear', 'electricalEquipment', 'fabricatedMetalProducts', 'foodAndDrink', 'glass', 'metals', 'organic', 'paperAndCardboard', 'petroleumProducts', 'water', 'cargoShip', 'freightFlights', 'freightTrain', 'hgvs', 'refrigeratedHgvs', 'seaTanker', 'vans', 'refuse', 'wasteWater']


In [115]:
printFormattedTypesForActivityL2(df[df["GHG Scopes and categories L1"] == "Scope 3"])


======= Activity L2 types per L1 =======


export const businessTravelFlights = [
'domesticAveragePassenger',
'longhaulInternational3700kmAveragePassenger',
'longhaulInternational3700kmBusinessClass',
'longhaulInternational3700kmEconomyClass',
'longhaulInternational3700kmFirstClass',
'longhaulInternational3700kmPremiumEconomyClass',
'shorthaulInternational3700kmAveragePassenger',
'shorthaulInternational3700kmBusinessClass',
'shorthaulInternational3700kmEconomyClass',
] as const;


export const businessTravelLandTravel = [
'busCoach',
'busLocal',
'carAverageElectric',
'carAverageIc',
'carLargeElectric',
'carLargeIc',
'carMediumElectric',
'carMediumIc',
'carSmallElectric',
'carSmallIc',
'metroAndTram',
'motorbikeAverage',
'motorbikeLarge',
'motorbikeMedium',
'motorbikeSmall',
'railInternational',
'railNational',
'taxiLarge',
'taxiRegular',
] as const;


export const businessTravelSeaTravel = [
'ferryAveragePassenger',
'ferryCarPassenger',
'ferryFootPassenger',
] as const;


export const

/var/folders/bh/rcxt7tsx5rv2pq74snstw_fr0000gn/T/ipykernel_46935/3314247233.py:65: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for activityL2 in df[df["GHG Scopes and categories L2"]== scopeType][df["Activity L1"] == activityL1]["Activity L2"].unique():


In [116]:
printSpreadOfActivityL1(df[df["GHG Scopes and categories L1"] == "Scope 3"])


======= Activity L1 spread =======

...businessTravelFlights,
...businessTravelLandTravel,
...businessTravelSeaTravel,
...employeeCommutingHomeOffice,
...employeeCommutingLandTravel,
...employeeCommutingSeaTravel,
...purchasedGoodsAndServicesBuildingMaterials,
...purchasedGoodsAndServicesClothingAndFootwear,
...purchasedGoodsAndServicesElectricalEquipment,
...purchasedGoodsAndServicesFabricatedMetalProducts,
...purchasedGoodsAndServicesFoodAndDrink,
...purchasedGoodsAndServicesGlass,
...purchasedGoodsAndServicesMetals,
...purchasedGoodsAndServicesOrganic,
...purchasedGoodsAndServicesPaperAndCardboard,
...purchasedGoodsAndServicesPetroleumProducts,
...purchasedGoodsAndServicesWater,
...upstreamTransportationAndDistributionCargoShip,
...upstreamTransportationAndDistributionFreightFlights,
...upstreamTransportationAndDistributionFreightTrain,
...upstreamTransportationAndDistributionHgvs,
...upstreamTransportationAndDistributionRefrigeratedHgvs,
...upstreamTransportationAndDistributionSea

In [117]:
printFormattedConstantsForSelectScopeType(df[df["GHG Scopes and categories L1"] == "Scope 3"])


======= Constants for select scope type =======

businessTravel:  {
    flights:  { 
    activityL2:  businessTravelFlights ,
    unitOfMeasurement: [...calculatedUnits,
pkm,
] as const, 
    },
    landTravel:  { 
    activityL2:  businessTravelLandTravel ,
    unitOfMeasurement: [...calculatedUnits,
pkm,
km,
miles,
] as const, 
    },
    seaTravel:  { 
    activityL2:  businessTravelSeaTravel ,
    unitOfMeasurement: [...calculatedUnits,
pkm,
] as const, 
    },
},
employeeCommuting:  {
    homeOffice:  { 
    activityL2:  employeeCommutingHomeOffice ,
    unitOfMeasurement: [...calculatedUnits,
FTEhour,
] as const, 
    },
    landTravel:  { 
    activityL2:  employeeCommutingLandTravel ,
    unitOfMeasurement: [...calculatedUnits,
pkm,
km,
miles,
] as const, 
    },
    seaTravel:  { 
    activityL2:  employeeCommutingSeaTravel ,
    unitOfMeasurement: [...calculatedUnits,
pkm,
] as const, 
    },
},
purchasedGoodsAndServices:  {
    buildingMaterials:  { 
    activityL2:  purcha

In [118]:
printMessagesForSelectScopeType(df, "Activity L1")


======= Messages for Activity L1 =======

'flights':  {
    defaultMessage: 'Flights' ,
    description: 'Label for Activity L1 flights' ,
},
'landTravel':  {
    defaultMessage: 'Land travel' ,
    description: 'Label for Activity L1 landTravel' ,
},
'seaTravel':  {
    defaultMessage: 'Sea travel' ,
    description: 'Label for Activity L1 seaTravel' ,
},
'homeOffice':  {
    defaultMessage: 'Home office' ,
    description: 'Label for Activity L1 homeOffice' ,
},
'buildingMaterials':  {
    defaultMessage: 'Building materials' ,
    description: 'Label for Activity L1 buildingMaterials' ,
},
'clothingAndFootwear':  {
    defaultMessage: 'Clothing and footwear' ,
    description: 'Label for Activity L1 clothingAndFootwear' ,
},
'electricalEquipment':  {
    defaultMessage: 'Electrical equipment' ,
    description: 'Label for Activity L1 electricalEquipment' ,
},
'fabricatedMetalProducts':  {
    defaultMessage: 'Fabricated metal products' ,
    description: 'Label for Activity L1 fabr

In [119]:
printMessagesForSelectScopeType(df, "Activity L2")


======= Messages for Activity L2 =======

'domesticAveragePassenger':  {
    defaultMessage: 'Domestic (Average passenger)' ,
    description: 'Label for Activity L2 domesticAveragePassenger' ,
},
'longhaulInternational3700kmAveragePassenger':  {
    defaultMessage: 'Long-haul (international >3700km) (Average passenger)' ,
    description: 'Label for Activity L2 longhaulInternational3700kmAveragePassenger' ,
},
'longhaulInternational3700kmBusinessClass':  {
    defaultMessage: 'Long-haul (international >3700km) (Business class)' ,
    description: 'Label for Activity L2 longhaulInternational3700kmBusinessClass' ,
},
'longhaulInternational3700kmEconomyClass':  {
    defaultMessage: 'Long-haul (international >3700km) (Economy class)' ,
    description: 'Label for Activity L2 longhaulInternational3700kmEconomyClass' ,
},
'longhaulInternational3700kmFirstClass':  {
    defaultMessage: 'Long-haul (international >3700km) (First class)' ,
    description: 'Label for Activity L2 longhaulInter

In [120]:
printCasesForMessagesHelpers(df, "Activity L1")


======= Messages for Activity L1 =======

case 'flights': 
    return activityL1Messages.flights;
case 'landTravel': 
    return activityL1Messages.landTravel;
case 'seaTravel': 
    return activityL1Messages.seaTravel;
case 'homeOffice': 
    return activityL1Messages.homeOffice;
case 'buildingMaterials': 
    return activityL1Messages.buildingMaterials;
case 'clothingAndFootwear': 
    return activityL1Messages.clothingAndFootwear;
case 'electricalEquipment': 
    return activityL1Messages.electricalEquipment;
case 'fabricatedMetalProducts': 
    return activityL1Messages.fabricatedMetalProducts;
case 'foodAndDrink': 
    return activityL1Messages.foodAndDrink;
case 'glass': 
    return activityL1Messages.glass;
case 'metals': 
    return activityL1Messages.metals;
case 'organic': 
    return activityL1Messages.organic;
case 'paperAndCardboard': 
    return activityL1Messages.paperAndCardboard;
case 'petroleumProducts': 
    return activityL1Messages.petroleumProducts;
case 'water':

In [121]:
printCasesForMessagesHelpers(df, "Activity L2")


======= Messages for Activity L2 =======

case 'domesticAveragePassenger': 
    return activityL2Messages.domesticAveragePassenger;
case 'longhaulInternational3700kmAveragePassenger': 
    return activityL2Messages.longhaulInternational3700kmAveragePassenger;
case 'longhaulInternational3700kmBusinessClass': 
    return activityL2Messages.longhaulInternational3700kmBusinessClass;
case 'longhaulInternational3700kmEconomyClass': 
    return activityL2Messages.longhaulInternational3700kmEconomyClass;
case 'longhaulInternational3700kmFirstClass': 
    return activityL2Messages.longhaulInternational3700kmFirstClass;
case 'longhaulInternational3700kmPremiumEconomyClass': 
    return activityL2Messages.longhaulInternational3700kmPremiumEconomyClass;
case 'shorthaulInternational3700kmAveragePassenger': 
    return activityL2Messages.shorthaulInternational3700kmAveragePassenger;
case 'shorthaulInternational3700kmBusinessClass': 
    return activityL2Messages.shorthaulInternational3700kmBusiness